# Dynamic Case Preparation

In [1]:
import numpy as np
import pandas as pd

import andes
import ams

In [2]:
%matplotlib inline

In [3]:
andes.config_logger(stream_level=30)
ams.config_logger(stream_level=30)

In [4]:
sp = ams.load('./../cases/IL200_opf.xlsx',
               setup=True, no_output=True,
               default_config=True)

In [5]:
stg_idx = sp.StaticGen.get_all_idxes()
stg_wt = sp.StaticGen.find_idx(keys='genfuel', values=['wind'], allow_all=True)[0]
stg_pv = sp.StaticGen.find_idx(keys='genfuel', values=['solar'], allow_all=True)[0]
stg_es = sp.StaticGen.find_idx(keys='genfuel', values=['ess'], allow_all=True)[0]

sn_wt = sp.StaticGen.get(src='Sn', attr='v', idx=stg_wt)
sn_pv = sp.StaticGen.get(src='Sn', attr='v', idx=stg_pv)
sn_es = sp.StaticGen.get(src='Sn', attr='v', idx=stg_es)
sn_total = sp.StaticGen.get(src='Sn', attr='v', idx=stg_idx)

print('Total generation: {0:.2f} MW'.format(sn_total.sum()))
print('Wind generation: {0:.2f} MW'.format(sn_wt.sum()))
print('Solar generation: {0:.2f} MW'.format(sn_pv.sum()))
print('ESS generation: {0:.2f} MW'.format(sn_es.sum()))

Total generation: 4251.60 MW
Wind generation: 1040.52 MW
Solar generation: 1916.70 MW
ESS generation: 36.72 MW


## Inspect Case

There are different types of generators:
- ST: Steam Turbine (includes nuclear, geothermal, and solar steam)
- NB: ST - Boiling Water Nuclear Reactor
- W2: Wind Turbine, Type 2
- GT: Combustion (Gas) Turbine
- PV: Photovoltaic
- ESS: Energy Storage System

In [6]:
stg_dfs = sp.StaticGen.as_df()
stg_cols = ['idx', 'bus', 'Sn', 'gentype', 'genfuel']

stg_df = stg_dfs[stg_cols]

stg_df

,idx,bus,Sn,gentype,genfuel
0,47,189,682.98,NB,nuclear
1,1,49,5.44,ST,coal
2,2,50,5.44,ST,coal
3,3,51,5.44,ST,coal
4,4,52,5.44,ST,coal
5,5,53,10.88,ST,coal
6,6,65,180.48,W2,wind
7,7,67,5.64,ST,coal
8,8,68,33.50,W2,wind
9,9,69,33.50,W2,wind


## Dynamic Models Replacement

In the original dynamic case, only ``GENROU`` is used for dynamic generators.
To better represent the dynamic behavior, following replacements are made:
1. W2 type gen, replace GENROU with: ``REGCA1``, ``REECA1``, ``REPCA1``, and ``WTDS``.
1. PV type gen, replace GENROU with: ``REGCA1``, ``REECA1``, ``REPCA1``.
1. ESS type gen, replace GENROU with ``ESD1``.

In [ ]:
# load the dynamics case but don't set up it
dyn_base_case = './../cases/IL200_dyn_new.xlsx'

sa = andes.load(dyn_base_case,
                setup=False, no_output=True,
                default_config=True)

# NOTE: bias is manually measured, in unit MW/0.1Hz
slack_bus = sa.Slack.bus.v[0]
sa.add('ACEc', param_dict=dict(bus=slack_bus,
                               bias=-45))

Generating code for 1 models on 12 processes.


'ACEc_1'

In [8]:
# In the ANDES case, get the StaticGen idx for type W2, NB, ST, GT, respectively
stg_idxes = sp.StaticGen.find_idx(keys='gentype',
                                  values=['W2', 'PV', 'ES'],
                                  allow_all=True)
stg_w2t = stg_idxes[0]
stg_pv = stg_idxes[1]
stg_ess = stg_idxes[2]

n_w2t = len(stg_w2t)
n_pv = len(stg_pv)
n_ess = len(stg_ess)

# In the ANDES case, get the corresponding SynGen idx
syg_w2t = sa.SynGen.find_idx(keys='gen', values=stg_w2t)
syg_pv = sa.SynGen.find_idx(keys='gen', values=stg_pv)
syg_ess = sa.SynGen.find_idx(keys='gen', values=stg_ess)

In [9]:
# --- GENROU -> WT2G: REGCA1 ---
REGCA1 = pd.DataFrame()

# mapped parameters
REGCA1['u'] = sa.SynGen.get(src='u', idx=syg_w2t)
REGCA1['idx'] = [f'WT_REGCA1_{i+1}' for i in range(n_w2t)]
REGCA1['bus'] = sa.SynGen.get(src='bus', idx=syg_w2t)
REGCA1['gen'] = sa.SynGen.get(src='gen', idx=syg_w2t)
REGCA1['Sn'] = sa.SynGen.get(src='Sn', idx=syg_w2t)
REGCA1['Lvplsw'] = [0] * n_w2t  # no voltage control
REGCA1['gammap'] = sa.SynGen.get(src='gammap', idx=syg_w2t)
REGCA1['gammaq'] = sa.SynGen.get(src='gammaq', idx=syg_w2t)

# unmapped parameters are skipped and default values are used
# add to the system
regca1_w2t = []
for row in REGCA1.itertuples(index=False):
    idx = sa.add(model='REGCA1', param_dict={**row._asdict()})
    regca1_w2t.append(idx)

# turn off the original GENROU
sa.SynGen.alter(src='u', value=0, idx=syg_w2t)

# --- GENROU -> WT2G: REECA1 ---
REECA1 = pd.DataFrame()

# mapped parameters
REECA1['u'] = [1] * n_w2t
REECA1['idx'] = [f'WT_REECA1_{i+1}' for i in range(n_w2t)]
REECA1['reg'] = regca1_w2t

# unmapped mandatory parameters
REECA1['PFFLAG'] = [0] * n_w2t
REECA1['VFLAG'] = [0] * n_w2t
REECA1['QFLAG'] = [0] * n_w2t
REECA1['PFLAG'] = [0] * n_w2t
REECA1['PQFLAG'] = [0] * n_w2t
REECA1['Vdip'] = [0.8] * n_w2t
REECA1['Vup'] = [1.2] * n_w2t
REECA1['Trv'] = [0.02] * n_w2t

REECA1['dbd1'] = [-0.1] * n_w2t
REECA1['dbd2'] = [0.1] * n_w2t
REECA1['Kqv'] = [20] * n_w2t
REECA1['Thld'] = [-2] * n_w2t
REECA1['Thld2'] = [1] * n_w2t
REECA1['Vref1'] = [1] * n_w2t
REECA1['Imax'] = [10] * n_w2t

REECA1['Kqp'] = [1] * n_w2t
REECA1['Kqi'] = [0] * n_w2t
REECA1['Kvp'] = [1] * n_w2t
REECA1['Kvi'] = [0] * n_w2t

# unmapped parameters are skipped and default values are used
# add to the system
reeca1_w2t = []
for row in REECA1.itertuples(index=False):
    idx = sa.add(model='REECA1', param_dict={**row._asdict()})
    reeca1_w2t.append(idx)

# --- GENROU -> WT2G: REPCA1 ---
REPCA1 = pd.DataFrame()

# mapped parameters
REPCA1['u'] = [1] * n_w2t
REPCA1['idx'] = [f'WT_REPCA1_{i+1}' for i in range(n_w2t)]
REPCA1['ree'] = reeca1_w2t
# manually assign mandatory param `line`
ree = REPCA1['ree']
reg = sa.REECA1.get(src='reg', attr='v', idx=ree)
bus = sa.REGCA1.get(src='bus', attr='v', idx=reg)
line1 = sa.Line.find_idx(keys='bus1', values=bus, allow_none=True, default=None)
line2 = sa.Line.find_idx(keys='bus2', values=bus, allow_none=True, default=None)
line = []
for l1, l2 in zip(line1, line2):
    if l1 is not None:
        line.append(l1)
    elif l2 is not None:
        line.append(l2)
    else:
        line.append(None)
REPCA1['line'] = line
REPCA1['VCFlag'] = [0] * n_w2t  # with droop
REPCA1['RefFlag'] = [0] * n_w2t  # Q control
REPCA1['Fflag'] = [0] * n_w2t  # Frequency control, disabled
REPCA1['PLflag'] = [0] * n_w2t  # Pline control, disabled

# unmapped parameters are skipped and default values are used
# add to the system
repca1_w2t = []
for row in REPCA1.itertuples(index=False):
    idx = sa.add(model='REPCA1', param_dict={**row._asdict()})
    repca1_w2t.append(idx)

# --- GENROU -> WT2G: WTDS, RenGovernor---
WTDS = pd.DataFrame()

# mapped parameters
WTDS['u'] = [1] * n_w2t
WTDS['ree'] = reeca1_w2t
WTDS['H'] = [3] * n_w2t
WTDS['D'] = [1] * n_w2t
WTDS['w0'] = [1] * n_w2t

wtds_w2t = []
for row in WTDS.itertuples(index=False):
    idx = sa.add(model='WTDS', param_dict={**row._asdict()})
    wtds_w2t.append(idx)

In [10]:
# --- GENROU -> PV: REGCA1 ---
REGCA1 = pd.DataFrame()

# mapped parameters
REGCA1['u'] = sa.SynGen.get(src='u', idx=syg_pv)
REGCA1['idx'] = [f'PV_REGCA1_{i+1}' for i in range(n_pv)]
REGCA1['bus'] = sa.SynGen.get(src='bus', idx=syg_pv)
REGCA1['gen'] = sa.SynGen.get(src='gen', idx=syg_pv)
REGCA1['Sn'] = sa.SynGen.get(src='Sn', idx=syg_pv)
REGCA1['Lvplsw'] = [0] * n_pv  # no voltage control
REGCA1['gammap'] = sa.SynGen.get(src='gammap', idx=syg_pv)
REGCA1['gammaq'] = sa.SynGen.get(src='gammaq', idx=syg_pv)

# unmapped parameters are skipped and default values are used
# add to the system
regca1_pv = []
for row in REGCA1.itertuples(index=False):
    idx = sa.add(model='REGCA1', param_dict={**row._asdict()})
    regca1_pv.append(idx)

# turn off the original GENROU
sa.SynGen.alter(src='u', value=0, idx=syg_pv)

# --- GENROU -> PV: REECA1 ---
REECA1 = pd.DataFrame()

# mapped parameters
REECA1['u'] = [1] * n_pv
REECA1['idx'] = [f'PV_REECA1_{i+1}' for i in range(n_pv)]
REECA1['reg'] = regca1_pv

# unmapped mandatory parameters
REECA1['PFFLAG'] = [0] * n_pv
REECA1['VFLAG'] = [0] * n_pv
REECA1['QFLAG'] = [0] * n_pv
REECA1['PFLAG'] = [0] * n_pv
REECA1['PQFLAG'] = [0] * n_pv
REECA1['Vdip'] = [0.8] * n_pv
REECA1['Vup'] = [1.2] * n_pv
REECA1['Trv'] = [0.02] * n_pv

REECA1['dbd1'] = [-0.1] * n_pv
REECA1['dbd2'] = [0.1] * n_pv
REECA1['Kqv'] = [20] * n_pv
REECA1['Thld'] = [-2] * n_pv
REECA1['Thld2'] = [1] * n_pv
REECA1['Vref1'] = [1] * n_pv
REECA1['Imax'] = [10] * n_pv

REECA1['Kqp'] = [1] * n_pv
REECA1['Kqi'] = [0] * n_pv
REECA1['Kvp'] = [1] * n_pv
REECA1['Kvi'] = [0] * n_pv

# unmapped parameters are skipped and default values are used
# add to the system
reeca1_pv = []
for row in REECA1.itertuples(index=False):
    idx = sa.add(model='REECA1', param_dict={**row._asdict()})
    reeca1_pv.append(idx)

# --- GENROU -> PV: REPCA1 ---
REPCA1 = pd.DataFrame()

# mapped parameters
REPCA1['u'] = [1] * n_pv
REPCA1['idx'] = [f'PV_REPCA1_{i+1}' for i in range(n_pv)]
REPCA1['ree'] = reeca1_pv
# manually assign mandatory param `line`
ree = REPCA1['ree']
reg = sa.REECA1.get(src='reg', attr='v', idx=ree)
bus = sa.REGCA1.get(src='bus', attr='v', idx=reg)
line1 = sa.Line.find_idx(keys='bus1', values=bus, allow_none=True, default=None)
line2 = sa.Line.find_idx(keys='bus2', values=bus, allow_none=True, default=None)
line = []
for l1, l2 in zip(line1, line2):
    if l1 is not None:
        line.append(l1)
    elif l2 is not None:
        line.append(l2)
    else:
        line.append(None)
REPCA1['line'] = line
REPCA1['VCFlag'] = [0] * n_pv  # with droop
REPCA1['RefFlag'] = [0] * n_pv  # Q control
REPCA1['Fflag'] = [0] * n_pv  # Frequency control, disabled
REPCA1['PLflag'] = [0] * n_pv  # Pline control, disabled

# unmapped parameters are skipped and default values are used
# add to the system
repca1_pv = []
for row in REPCA1.itertuples(index=False):
    idx = sa.add(model='REPCA1', param_dict={**row._asdict()})
    repca1_pv.append(idx)

In [11]:
# --- GENROU -> ESS: ESD1 ---
ESD1 = pd.DataFrame()

# mapped parameters
ESD1['u'] = sa.SynGen.get(src='u', idx=syg_ess)
ESD1['bus'] = sa.SynGen.get(src='bus', idx=syg_ess)
ESD1['gen'] = sa.SynGen.get(src='gen', idx=syg_ess)
ESD1['Sn'] = sa.SynGen.get(src='Sn', idx=syg_ess)
ESD1['En'] = sa.SynGen.get(src='Sn', idx=syg_ess)
ESD1['pqflag'] = [1] * n_ess  # P priority
ESD1['qmx'] = [999] * n_ess
ESD1['qmn'] = [-999] * n_ess
ESD1['pmx'] = [999] * n_ess
ESD1['fdbd'] = [0] * n_ess  # frequency deviation deadband, Hz
ESD1['ddn'] = [0] * n_ess  # gain after f deadband
ESD1['ialim'] = [999] * n_ess
ESD1['gammap'] = sa.SynGen.get(src='gammap', idx=syg_ess)
ESD1['gammaq'] = sa.SynGen.get(src='gammaq', idx=syg_ess)

# unmapped parameters are skipped and default values are used
# add to the system
esd1_ess = []
for row in ESD1.itertuples(index=False):
    idx = sa.add(model='ESD1', param_dict={**row._asdict()})
    esd1_ess.append(idx)

# turn off the original GENROU
sa.SynGen.alter(src='u', value=0, idx=syg_ess)

True

In [12]:
sa.setup()

# relax TurbineGov upper limit
vmax0 = sa.TGOV1NDB.get(src='VMAX', attr='v', idx=sa.TGOV1NDB.idx.v)
sa.TGOV1NDB.set(src='VMAX', attr='v', idx=sa.TGOV1NDB.idx.v,
                value=100 * vmax0)

True

In [13]:
sa.PFlow.run()
_ = sa.TDS.init()

GENROU (vf range) out of typical lower limit.

   idx     | values | limit
-----------+--------+------
 GENROU_7  | 0.865  | 1    
 GENROU_38 | 0.977  | 1    


/Users/jinningwang/work/andes/andes/core/model/model.py:790: ComplexWarning: Casting complex values to real discards the imaginary part
  instance.v = np.array(func(*self.s_args[name]),
/Users/jinningwang/work/andes/andes/core/model/model.py:829: ComplexWarning: Casting complex values to real discards the imaginary part
  instance.v[:] = func(*self.s_args[name])


In [14]:
# Export to XLSX for Further Use
andes.io.xlsx.write(sa, './../cases/IL200_dyn_db.xlsx',
                    overwrite=True)

True